# torch.save(obj, f, pickle_module=pickle, pickle_protocol=2, _use_new_zipfile_serialization=True)

## Тодорхойлолт
- **torch.save()** нь PyTorch объектуудыг файлд хадгалах функцийн бөгөөд загварын параметрүүд, оптимизаторын төлөв, бусад PyTorch объектуудыг сериалчлахад зориулагдсан.

## Параметрүүд

- **obj:** Хадгалах объект (модел, тензор, optimizer, гэх мэт)

- **f:** Файл замын string эсвэл file-like объект

- **pickle_module (optional):** Ашиглах pickle модуль. Анхны утга: pickle

- **pickle_protocol (optional):** Pickle протоколын дугаар. Анхны утга: 2

- **_use_new_zipfile_serialization (optional):** Шинэ zipfile сериалчлал ашиглах эсэх. Анхны утга: True

## Үндсэн хэрэглээ

### 1. Энгийн тензор хадгалах

In [1]:
import torch

# Тензор үүсгэх
tensor = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0])
print(f"Тензор: {tensor}")
print(f"Тензорын хэлбэр: {tensor.shape}")

# Тензор хадгалах
torch.save(tensor, 'tensor.pth')
print("Тензор 'tensor.pth' файлд хадгалагдлаа")

# Тензор дуурайх
loaded_tensor = torch.load('tensor.pth')
print(f"Дуурайсан тензор: {loaded_tensor}")
print(f"Тэнцүү байна уу? {torch.equal(tensor, loaded_tensor)}")

Тензор: tensor([1., 2., 3., 4., 5.])
Тензорын хэлбэр: torch.Size([5])
Тензор 'tensor.pth' файлд хадгалагдлаа
Дуурайсан тензор: tensor([1., 2., 3., 4., 5.])
Тэнцүү байна уу? True


### 2. Моделийн параметр хадгалах

In [2]:
import torch.nn as nn

# Энгийн загвар үүсгэх
model = nn.Sequential(
    nn.Linear(10, 5),
    nn.ReLU(),
    nn.Linear(5, 2)
)

# Загварын параметрүүдийг хадгалах
torch.save(model.state_dict(), 'model_params.pth')
print("Моделийн параметрүүд хадгалагдлаа")

# Параметрийн хэмжээг харах
print(f"Параметрийн тоо: {sum(p.numel() for p in model.parameters())}")

# Шинэ загварт дуурайх
new_model = nn.Sequential(
    nn.Linear(10, 5),
    nn.ReLU(),
    nn.Linear(5, 2)
)

new_model.load_state_dict(torch.load('model_params.pth'))
print("Шинэ загварт параметрүүд дуурайлагдлаа")

Моделийн параметрүүд хадгалагдлаа
Параметрийн тоо: 67
Шинэ загварт параметрүүд дуурайлагдлаа


## Деталь тайлбар

### 1. Өөр өөр төрлийн объектууд

In [ ]:
# Өөр өөр төрлийн PyTorch объектууд хадгалах

# 1. Тензор
tensor = torch.randn(3, 4)

# 2. Модел
model = nn.Linear(5, 3)

# 3. Оптимизатор
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 4. Словарь
data_dict = {
    'accuracy': 0.95,
    'loss_history': [0.5, 0.3, 0.2, 0.1],
    'config': {'batch_size': 32, 'epochs': 100}
}

# 5. Бүх объектуудыг нэгтгэх
save_data = {
    'tensor': tensor,
    'model_state': model.state_dict(),
    'optimizer_state': optimizer.state_dict(),
    'metadata': data_dict,
    'model_class': type(model)  # Моделийн төрөл ч хадгалж болно
}

# Нэг файлд хадгалах
torch.save(save_data, 'all_data.pth')
print("Бүх өгөгдөл хадгалагдлаа")

# Дуурайх - recommended: load with weights_only=True (safe, default in newer PyTorch)
loaded_data = torch.load('all_data.pth', weights_only=False)
print(f"Дуурайсан өгөгдлийн түлхүүрүүд: {list(loaded_data.keys())}")

# If you trust the checkpoint source and need to unpickle classes, use one of the trusted alternatives:
# 1) Use weights_only=False (can execute arbitrary code; only for trusted files):
# loaded_data = torch.load('all_data.pth', weights_only=False)
#
# 2) Or allowlist specific globals before loading (safer than blanket unpickling):
# import torch.serialization
# torch.serialization.add_safe_globals([torch.nn.modules.linear.Linear])
# loaded_data = torch.load('all_data.pth', weights_only=False)
#
# Note: we stored class module/name so you can reconstruct the class safely if needed.

Бүх өгөгдөл хадгалагдлаа
Дуурайсан өгөгдлийн түлхүүрүүд: ['tensor', 'model_state', 'optimizer_state', 'metadata', 'model_class']


### 2. Pickle протоколын сонголт

In [4]:
import pickle
import os

# Өөр өөр pickle протокол ашиглах
data = {'tensor': torch.randn(2, 3), 'value': 42}

# Протокол 2 (Python 2-той нийцдэг)
torch.save(data, 'protocol2.pth', pickle_protocol=2)

# Протокол 4 (Python 3.4+)
torch.save(data, 'protocol4.pth', pickle_protocol=4)

# Протокол 5 (Python 3.8+, илүү их файлыг дэмждэг)
torch.save(data, 'protocol5.pth', pickle_protocol=5)

# Файлын хэмжээг харьцуулах

print(f"Протокол 2 файлын хэмжээ: {os.path.getsize('protocol2.pth')} байт")
print(f"Протокол 4 файлын хэмжээ: {os.path.getsize('protocol4.pth')} байт")
print(f"Протокол 5 файлын хэмжээ: {os.path.getsize('protocol5.pth')} байт")

Протокол 2 файлын хэмжээ: 1655 байт
Протокол 4 файлын хэмжээ: 1591 байт
Протокол 5 файлын хэмжээ: 1591 байт


### 3. Шинэ zipfile сериалчлал

In [5]:
# Том тензор үүсгэх
large_tensor = torch.randn(1000, 1000, 100)  # 400MB орчим

# Шинэ zipfile сериалчлал ашиглах (анхдагч)
torch.save({'tensor': large_tensor}, 'new_serialization.pth', 
           _use_new_zipfile_serialization=True)

# Хуучин сериалчлал ашиглах
torch.save({'tensor': large_tensor}, 'old_serialization.pth',
           _use_new_zipfile_serialization=False)

# Файлын хэмжээг харьцуулах
print(f"Шинэ сериалчлалын файлын хэмжээ: {os.path.getsize('new_serialization.pth') / 1024**2:.2f} MB")
print(f"Хуучин сериалчлалын файлын хэмжээ: {os.path.getsize('old_serialization.pth') / 1024**2:.2f} MB")

# Хүчин чармайлт
import time

start = time.time()
torch.save({'tensor': large_tensor}, 'test_new.pth', _use_new_zipfile_serialization=True)
new_time = time.time() - start

start = time.time()
torch.save({'tensor': large_tensor}, 'test_old.pth', _use_new_zipfile_serialization=False)
old_time = time.time() - start

print(f"\nШинэ сериалчлалын хугацаа: {new_time:.2f} секунд")
print(f"Хуучин сериалчлалын хугацаа: {old_time:.2f} секунд")

Шинэ сериалчлалын файлын хэмжээ: 381.47 MB
Хуучин сериалчлалын файлын хэмжээ: 381.47 MB

Шинэ сериалчлалын хугацаа: 19.75 секунд
Хуучин сериалчлалын хугацаа: 17.68 секунд


## Практик хэрэглээ
### 1. Бүтэн сургалтын checkpoint

In [10]:
import torch.optim as optim

class TrainingCheckpoint:
    """Бүрэн сургалтын checkpoint хадгалах"""
    
    def create_checkpoint(self, model, optimizer, epoch, loss, other_info=None):
        """Checkpoint үүсгэх"""
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            'model_class': type(model),
            'optimizer_class': type(optimizer),
            'timestamp': '2024-01-15 10:30:00'
        }
        
        if other_info:
            checkpoint.update(other_info)
        
        return checkpoint
    
    def save_checkpoint(self, checkpoint, filename):
        """Checkpoint хадгалах"""
        torch.save(checkpoint, filename)
        print(f"Checkpoint хадгаллаа: {filename}")
    
    def load_checkpoint(self, filename, model=None, optimizer=None):
        """Checkpoint дуурайх"""
        # Recommended: load with weights_only=True to avoid unpickling arbitrary globals
        checkpoint = torch.load(filename, weights_only=False)
        
        if model is not None:
            model.load_state_dict(checkpoint['model_state_dict'])
        
        if optimizer is not None:
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        
        print(f"Checkpoint дуурайлаа: epoch {checkpoint['epoch']}, loss {checkpoint['loss']:.4f}")
        
        return checkpoint

# Туршилт
model = nn.Linear(10, 2)
optimizer = optim.Adam(model.parameters(), lr=0.001)

checkpoint_manager = TrainingCheckpoint()

# Checkpoint үүсгэх
checkpoint = checkpoint_manager.create_checkpoint(
    model=model,
    optimizer=optimizer,
    epoch=10,
    loss=0.1234,
    other_info={'accuracy': 0.95, 'batch_size': 32}
)

# Хадгалах
checkpoint_manager.save_checkpoint(checkpoint, 'training_checkpoint.pth')

# Дуурайх
loaded_checkpoint = checkpoint_manager.load_checkpoint(
    'training_checkpoint.pth', 
    model=model, 
    optimizer=optimizer
    
)

Checkpoint хадгаллаа: training_checkpoint.pth
Checkpoint дуурайлаа: epoch 10, loss 0.1234
